In [163]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from fluid_properties.utils import scrape_nist_isotherm, scrape_nist_satcurve

FLUID = 'CO2'
TEMPERATURE_UPSTREAM_RANGE = np.arange(14, 14.11, 0.01)  # degC
TEMPERATURE_UPSTREAM = 15  # degC
PRESSURE_UPSTREAM = 46  # bar

In [89]:
df_sat = scrape_nist_satcurve(
    fluid='CO2', pressure_range=np.arange(0.01835, PRESSURE_UPSTREAM + 0.1, 0.1))
df = scrape_nist_isotherm(fluid=FLUID, isotherm=TEMPERATURE_UPSTREAM,
                          pressure_range=np.arange(0.01835, PRESSURE_UPSTREAM, 0.1))

df['Pressure Upstream (bar)'] = PRESSURE_UPSTREAM
df['Pressure Drop (bar)'] = df['Pressure Upstream (bar)'] - \
    df['Pressure (bar)']
df['Pressure Downstream (bar)'] = df['Pressure (bar)']

df['Temperature Upstream (C)'] = TEMPERATURE_UPSTREAM
df['Temperature Drop (C)'] = df['Joule-Thomson (K/bar)'] * \
    df['Pressure Drop (bar)']
df['Temperature Downstream (C)'] = df['Temperature Upstream (C)'] - \
    df['Temperature Drop (C)']

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_sat['Temperature (C)'],
              y=df_sat['Pressure (bar)'], name='Liquid-Vapor Saturation Curve'))
fig.add_trace(go.Scatter(x=df['Temperature Downstream (C)'], y=df['Pressure Downstream (bar)'],
              name=f'Upstream Temperature = {TEMPERATURE_UPSTREAM} C'))
fig.update_layout(template='plotly_dark', paper_bgcolor='#1f1f1f',
                  xaxis_title='Temperature (C)', yaxis_title='Pressure (bar)')


In [3]:
df_sat = df_sat[['Temperature (C)', 'Pressure (bar)']]
df_sat = pd.concat([df_sat, pd.DataFrame({'Temperature (C)': np.linspace(11, 60, 50), 'Pressure (bar)': 46})], ignore_index=True)

In [164]:
from tqdm import tqdm

df_dict = {}
for temperature_upstream in tqdm(TEMPERATURE_UPSTREAM_RANGE):
    df = scrape_nist_isotherm(fluid='CO2', isotherm=temperature_upstream,
                              pressure_range=np.arange(0.01835, PRESSURE_UPSTREAM, 0.1))

    df['Pressure Upstream (bar)'] = PRESSURE_UPSTREAM
    df['Pressure Drop (bar)'] = df['Pressure Upstream (bar)'] - \
        df['Pressure (bar)']
    df['Pressure Downstream (bar)'] = df['Pressure (bar)']

    df['Temperature Upstream (C)'] = temperature_upstream
    df['Temperature Drop (C)'] = df['Joule-Thomson (K/bar)'] * \
        df['Pressure Drop (bar)']
    df['Temperature Downstream (C)'] = df['Temperature Upstream (C)'] - \
        df['Temperature Drop (C)']

    df_dict[temperature_upstream] = df


# fig = go.Figure()
# fig.add_trace(go.Scatter(x=df_sat['Temperature (C)'],
#               y=df_sat['Pressure (bar)'], name='Liquid-Vapor Phase Boundary', stackgroup='one'))
# for temperature_upstream in TEMPERATURE_UPSTREAM_RANGE:
#     df = df_dict[temperature_upstream]
#     fig.add_trace(go.Scatter(x=df['Temperature Downstream (C)'], y=df['Pressure Downstream (bar)'],
#                   name=f'Upstream Temperature = {temperature_upstream} C'))
# fig.update_layout(template='plotly_dark', paper_bgcolor='#1f1f1f',
#                   xaxis_title='Temperature (C)', yaxis_title='Pressure (bar)')
# fig.show()


100%|██████████| 11/11 [00:15<00:00,  1.45s/it]


In [165]:
rows = []

for temperature_upstream in df_dict:
    df = df_dict[temperature_upstream]
    temp = np.expand_dims(df['Temperature Downstream (C)'], axis=1)
    press = np.expand_dims(df['Pressure Downstream (bar)'], axis=1)

    temp_sat = np.expand_dims(df_sat['Temperature (C)'], axis=1)
    press_sat = np.expand_dims(df_sat['Pressure (bar)'], axis=1)

    dtemp = temp - temp_sat.T
    dpress = press - press_sat.T

    distances = np.sqrt(dtemp**2 + dpress**2)

    min_index = np.argmin(distances)
    i_min, j_min = np.unravel_index(min_index, distances.shape)

    # Append the values as a tuple or list to the rows list
    rows.append([temperature_upstream, distances[i_min, j_min],
                dtemp[i_min, j_min], dpress[i_min, j_min]])

# Convert the accumulated rows to a DataFrame
df_distances = pd.DataFrame(
    rows, columns=['upstream temperature', 'distances', 'dtemp', 'dpress'])

In [166]:
df_distances

,upstream temperature,distances,dtemp,dpress
0,14.00,0.038039,-0.001723,0.038
1,14.01,0.039120,0.009294,0.038
2,14.02,0.042849,0.017750,0.039
3,14.03,0.048815,0.029358,0.039
4,14.04,0.057026,0.041605,0.039
5,14.05,0.061189,0.004809,-0.061
6,14.06,0.061046,-0.002365,-0.061
7,14.07,0.061016,-0.001408,-0.061
8,14.08,0.061040,-0.002219,-0.061
9,14.09,0.061023,0.001661,-0.061
